In [5]:
# =============================================================================
# IFAE (Income-Health-Access Equity) ZIP/ZCTA Ranking — Notebook Version
# -----------------------------------------------------------------------------
# Inputs (all CSVs; robust loaders handle small format differences):
#   FINANCIAL_CSV  -> needs: NAME (ZCTA/ZIP name or code), S1901_C01_012E (median HH income)
#   HEALTH_CSV     -> needs: ZCTA5 (5-digit), GHLTH_CrudePrev (general poor health %)
#   PHARMACY_CSV   -> needs: ZIP (5-digit), NAME (pharmacy name), X, Y (coords; optional)
#   POPULATION_CSV -> teammate used skiprows=10; script will autodetect 3 useful cols:
#                     ZCTA/ZIP code, population, and optionally land area (km2 or mi2)
#
# Outputs:
#   results/national_ifae_rank.csv
#   results/top5_ifae.csv
#
# Method:
#   - Build features by ZCTA/ZIP: median income, poor health %, population,
#     pharmacies per 10k population (as access proxy), and (if available) density
#   - Normalize features (0..1), form a transparent composite, and also fit
#     an IsolationForest on the feature vector. Final IFAE score = 0.5*composite +
#     0.5*iforest_anomaly (higher => more underserved / priority).
#
# Dependencies: pandas, numpy, scikit-learn (IsolationForest); pyarrow optional.
# =============================================================================

import os, time
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd

from sklearn.ensemble import IsolationForest

# === CONFIG: set your file paths here ===
FINANCIAL_CSV  = "data/financial_data.csv"     # needs: NAME, S1901_C01_012E
HEALTH_CSV     = "data/health_data.csv"        # needs: ZCTA5, GHLTH_CrudePrev
PHARMACY_CSV   = "data/pharmacy_data.csv"      # needs: ZIP, NAME, X, Y
POPULATION_CSV = "data/population_data.csv"    # your teammate used skiprows=10; 3 useful cols

OUT_DIR        = "results"                     # outputs: national_ifae_rank.csv, top5_ifae.csv
CONTAMINATION  = 0.03                          # IsolationForest sensitivity (try 0.02–0.05)
TOP_K          = 10

# ---------- Monitoring helpers ----------
_T0 = time.time()
def stamp(msg):
    now = datetime.now().strftime("%H:%M:%S")
    print(f"[{now} +{time.time()-_T0:6.2f}s] {msg}", flush=True)

class Step:
    def __init__(self, name): self.name=name; self.t0=None
    def __enter__(self): self.t0=time.time(); stamp(f"▶ {self.name} ..."); return self
    def __exit__(self, et, ev, tb):
        dt = time.time()-self.t0
        stamp(("✓ " if et is None else "✖ ") + f"{self.name} {'done' if et is None else 'failed'} in {dt:.2f}s")

def read_csv_smart(path, **kw):
    p = Path(path)
    if not p.exists(): raise FileNotFoundError(f"Missing file: {path}")
    try:
        return pd.read_csv(path, low_memory=False, **kw)
    except Exception as e:
        stamp(f"CSV read warning: {e}. Retrying with simpler defaults.")
        return pd.read_csv(path)

def minmax(s):
    x = pd.to_numeric(s, errors='coerce')
    mn, mx = x.min(skipna=True), x.max(skipna=True)
    if np.isnan(mn) or np.isnan(mx) or mx==mn:
        return pd.Series(0.0, index=x.index)
    return (x - mn) / (mx - mn)

def coerce_zcta(series):
    """Return 5-digit, zero-padded strings; strips non-digit chars if present."""
    s = series.astype(str).str.extract(r"(\d{3,5})", expand=False)
    return s.fillna("").str.zfill(5)

# ---------- Load & prepare each source ----------
with Step("Load FINANCIAL (income)"):
    fin = read_csv_smart(FINANCIAL_CSV)
    # Try to identify ZCTA code
    zcta_col = None
    for cand in ["ZCTA5","zcta5","ZCTA","zcta","ZIP","Zip","NAME","Name","name"]:
        if cand in fin.columns:
            zcta_col = cand; break
    if zcta_col is None:
        raise KeyError(f"Could not find a ZCTA/ZIP column in {FINANCIAL_CSV}. Columns: {list(fin.columns)[:10]} ...")
    if "S1901_C01_012E" not in fin.columns:
        raise KeyError("Missing S1901_C01_012E (median HH income) in FINANCIAL_CSV.")
    fin = fin.rename(columns={zcta_col:"ZCTA5"})
    fin["ZCTA5"] = coerce_zcta(fin["ZCTA5"])
    fin = fin[["ZCTA5","S1901_C01_012E"]].copy()
    fin["S1901_C01_012E"] = pd.to_numeric(fin["S1901_C01_012E"], errors="coerce")
    stamp(f"FIN rows={len(fin)}, null income={fin['S1901_C01_012E'].isna().sum()}")

with Step("Load HEALTH (poor general health %)"):
    hlth = read_csv_smart(HEALTH_CSV)
    if "ZCTA5" not in hlth.columns:
        # attempt alias
        for cand in ["ZCTA","zcta","ZIP","Zip","name","NAME"]:
            if cand in hlth.columns:
                hlth = hlth.rename(columns={cand:"ZCTA5"})
                break
    if "ZCTA5" not in hlth.columns or "GHLTH_CrudePrev" not in hlth.columns:
        raise KeyError("HEALTH_CSV must contain ZCTA5 and GHLTH_CrudePrev.")
    hlth["ZCTA5"] = coerce_zcta(hlth["ZCTA5"])
    hlth["GHLTH_CrudePrev"] = pd.to_numeric(hlth["GHLTH_CrudePrev"], errors="coerce")
    hlth = hlth[["ZCTA5","GHLTH_CrudePrev"]]
    stamp(f"HEALTH rows={len(hlth)}, null health%={hlth['GHLTH_CrudePrev'].isna().sum()}")

with Step("Load PHARMACY (access proxy)"):
    ph = read_csv_smart(PHARMACY_CSV)
    for needed in ["ZIP","NAME"]:
        if needed not in ph.columns:
            raise KeyError(f"PHARMACY_CSV must contain {needed}.")
    ph["ZCTA5"] = coerce_zcta(ph["ZIP"])
    # count unique pharmacies per ZCTA
    ph_cnt = ph.groupby("ZCTA5", dropna=False)["NAME"].nunique(dropna=True).reset_index()
    ph_cnt = ph_cnt.rename(columns={"NAME":"pharmacies_count"})
    stamp(f"PHARMACY unique ZCTA5={ph_cnt['ZCTA5'].nunique()}, total pharmacies={ph_cnt['pharmacies_count'].sum()}")

with Step("Load POPULATION (skiprows=10 heuristic + autodetect)"):
    pop = read_csv_smart(POPULATION_CSV, skiprows=10)
    # try to identify code + population + (optional) land area
    code_col = None
    for cand in ["ZCTA5","ZCTA","ZIP","Zip","GEOID","geoid","NAME","name"]:
        if cand in pop.columns:
            code_col=cand; break
    if code_col is None:
        # fallback: choose first object-like column
        obj_cols = [c for c in pop.columns if pop[c].dtype == object]
        code_col = obj_cols[0] if obj_cols else pop.columns[0]
    pop = pop.rename(columns={code_col:"ZCTA5"})
    pop["ZCTA5"] = coerce_zcta(pop["ZCTA5"])

    # population column guess
    pop_col = None
    for cand in ["POP","Population","population","TOTAL_POP","TotPop","DP05_0001E","pop"]:
        if cand in pop.columns:
            pop_col=cand; break
    if pop_col is None:
        # choose the largest-sum numeric column as population
        num_cols = [c for c in pop.columns if pd.api.types.is_numeric_dtype(pop[c])]
        sums = {c: pd.to_numeric(pop[c], errors="coerce").sum(skipna=True) for c in num_cols}
        pop_col = max(sums, key=sums.get) if sums else None
    if pop_col is None:
        raise KeyError("Could not infer population column in POPULATION_CSV.")

    # optional land area
    land_col = None
    for cand in ["land_area_km2","Land_Area_km2","ALAND_KM2","aland_km2","ALAND","area","AREA_KM2","ALAND_SQMI","AREALAND"]:
        if cand in pop.columns:
            land_col = cand; break

    keep_cols = ["ZCTA5", pop_col] + ([land_col] if land_col else [])
    pop = pop[keep_cols].copy()
    pop = pop.rename(columns={pop_col:"population"})
    pop["population"] = pd.to_numeric(pop["population"], errors="coerce")
    if land_col:
        pop = pop.rename(columns={land_col:"land_area_raw"})
        pop["land_area_raw"] = pd.to_numeric(pop["land_area_raw"], errors="coerce")

    stamp(f"POP rows={len(pop)}, null pop={pop['population'].isna().sum()}, land_col={land_col}")

# ---------- Merge to ZCTA frame ----------
with Step("Merge all features by ZCTA/ZIP"):
    df = fin.merge(hlth, on="ZCTA5", how="outer") \
            .merge(pop, on="ZCTA5", how="outer") \
            .merge(ph_cnt, on="ZCTA5", how="left")
    stamp(f"Merged shape: {df.shape}")
    print("Top null counts:\n", df.isna().sum().sort_values(ascending=False).head(10).to_string(), flush=True)

# ---------- Feature engineering ----------
with Step("Feature engineering"):
    # income
    df["median_income"] = pd.to_numeric(df["S1901_C01_012E"], errors="coerce")
    # health burden
    df["poor_health_pct"] = pd.to_numeric(df["GHLTH_CrudePrev"], errors="coerce")
    # access proxy
    df["pharmacies_count"] = pd.to_numeric(df["pharmacies_count"], errors="coerce").fillna(0)
    # pharmacies per 10k (guard against div by zero)
    df["pharm_per_10k"] = np.where(df["population"].gt(0),
                                   1e4 * df["pharmacies_count"] / df["population"],
                                   np.nan)
    # population density if land provided; try to detect units:
    if "land_area_raw" in df.columns:
        # If raw area looks like square meters (ALAND), convert to km2
        # Heuristic: median ~10^6–10^9 for sq meters; <1e4 for km2
        med_area = np.nanmedian(df["land_area_raw"])
        if np.isfinite(med_area) and med_area > 1e5:
            df["land_area_km2"] = df["land_area_raw"] / 1e6
        else:
            df["land_area_km2"] = df["land_area_raw"]
        df["pop_density"] = np.where(df["land_area_km2"].gt(0),
                                     df["population"] / df["land_area_km2"],
                                     np.nan)
    else:
        df["pop_density"] = np.nan

    # Normalizations
    df["income_norm_inv"]   = 1.0 - minmax(df["median_income"])         # lower income -> higher need
    df["health_norm"]       = minmax(df["poor_health_pct"])             # higher poor-health% -> higher need
    df["access_norm_inv"]   = 1.0 - minmax(df["pharm_per_10k"])         # fewer pharmacies/10k -> higher need
    df["pop_norm"]          = minmax(df["population"])                  # prioritize where more people
    df["density_norm"]      = minmax(df["pop_density"])                 # optional

    for col in ["median_income","poor_health_pct","pharm_per_10k","population","pop_density"]:
        if col in df.columns:
            x = pd.to_numeric(df[col], errors="coerce")
            stamp(f"Range {col}: min={np.nanmin(x):.3f} max={np.nanmax(x):.3f} NaN={np.isnan(x).sum()}")

# ---------- Composite score ----------
with Step("Compute composite & train IsolationForest"):
    # Transparent composite (tweakable weights)
    w_income, w_health, w_access, w_pop, w_density = 0.30, 0.35, 0.25, 0.08, 0.02
    df["composite"] = (
        w_income  * df["income_norm_inv"].fillna(0) +
        w_health  * df["health_norm"].fillna(0) +
        w_access  * df["access_norm_inv"].fillna(0) +
        w_pop     * df["pop_norm"].fillna(0) +
        w_density * df["density_norm"].fillna(0)
    )

    # IsolationForest on normalized features
    feats = df[["income_norm_inv","health_norm","access_norm_inv","pop_norm","density_norm"]].fillna(0.0).values
    iforest = IsolationForest(random_state=42, contamination=CONTAMINATION, n_estimators=300)
    iforest.fit(feats)
    # decision_function: higher is less anomalous; we want higher = more underserved,
    # so invert and min-max
    decision = iforest.decision_function(feats)  # larger -> more normal
    inv = -decision
    # scale to 0..1
    inv = (inv - inv.min()) / (inv.max() - inv.min() + 1e-12)
    df["iforest_anomaly"] = inv

    # Final IFAE score = blend
    df["IFAE_score"] = 0.5 * df["composite"] + 0.5 * df["iforest_anomaly"]

with Step("Finalize outputs"):
    out = Path(OUT_DIR); out.mkdir(parents=True, exist_ok=True)

    # Prepare nice columns
    keep = [
        "ZCTA5","IFAE_score","composite","iforest_anomaly",
        "median_income","poor_health_pct","population","pharmacies_count","pharm_per_10k",
        "income_norm_inv","health_norm","access_norm_inv","pop_norm","density_norm","pop_density"
    ]
    keep = [c for c in keep if c in df.columns]
    ranked = df[keep].copy()
    ranked = ranked.sort_values("IFAE_score", ascending=False).reset_index(drop=True)

    # Write full national ranking & top-k
    national_path = out / "national_ifae_rank.csv"
    topk_path     = out / "top5_ifae.csv"
    ranked.to_csv(national_path, index=False)
    ranked.tail(TOP_K).to_csv(topk_path, index=False)

    stamp(f"Wrote {national_path}")
    stamp(f"Wrote {topk_path}")
    print("\nTop 5 preview:\n", ranked.tail(TOP_K).to_string(index=False))

stamp("All done. 🚀")


[20:38:42 +  0.00s] ▶ Load FINANCIAL (income) ...
[20:38:42 +  0.30s] FIN rows=33773, null income=3264
[20:38:42 +  0.30s] ✓ Load FINANCIAL (income) done in 0.30s
[20:38:42 +  0.30s] ▶ Load HEALTH (poor general health %) ...
[20:38:42 +  0.51s] HEALTH rows=32520, null health%=0
[20:38:42 +  0.51s] ✓ Load HEALTH (poor general health %) done in 0.21s
[20:38:42 +  0.51s] ▶ Load PHARMACY (access proxy) ...
[20:38:43 +  1.02s] PHARMACY unique ZCTA5=14940, total pharmacies=61970
[20:38:43 +  1.02s] ✓ Load PHARMACY (access proxy) done in 0.50s
[20:38:43 +  1.02s] ▶ Load POPULATION (skiprows=10 heuristic + autodetect) ...
[20:38:43 +  1.06s] POP rows=40959, null pop=0, land_col=None
[20:38:43 +  1.06s] ✓ Load POPULATION (skiprows=10 heuristic + autodetect) done in 0.05s
[20:38:43 +  1.07s] ▶ Merge all features by ZCTA/ZIP ...
[20:38:43 +  1.10s] Merged shape: (41094, 5)
Top null counts:
 pharmacies_count    26171
S1901_C01_012E      10585
GHLTH_CrudePrev      8574
population            135
ZCT

/var/folders/9d/5gzqwh3d5sx43n9fsy97czyc0000gn/T/ipykernel_54177/3424445507.py:210: RuntimeWarning: All-NaN axis encountered
  stamp(f"Range {col}: min={np.nanmin(x):.3f} max={np.nanmax(x):.3f} NaN={np.isnan(x).sum()}")


[20:38:44 +  1.79s] ✓ Compute composite & train IsolationForest done in 0.69s
[20:38:44 +  1.79s] ▶ Finalize outputs ...
[20:38:44 +  1.98s] Wrote results/national_ifae_rank.csv
[20:38:44 +  1.98s] Wrote results/top5_ifae.csv

Top 5 preview:
 ZCTA5  IFAE_score  composite  iforest_anomaly  median_income  poor_health_pct  population  pharmacies_count  pharm_per_10k  income_norm_inv  health_norm  access_norm_inv  pop_norm  density_norm  pop_density
96854    0.195164   0.250390         0.139938            NaN              NaN       599.0               0.0            0.0              NaN          NaN              1.0  0.004877           0.0          NaN
65636    0.195164   0.250390         0.139938            NaN              NaN       599.0               0.0            0.0              NaN          NaN              1.0  0.004877           0.0          NaN
39098    0.195164   0.250390         0.139938            NaN              NaN       598.0               0.0            0.0              

In [1]:



# =============================================================================
# IFAE (Income-Health-Access Equity) ZIP/ZCTA Ranking — Notebook Version
# -----------------------------------------------------------------------------
# Inputs (all CSVs; robust loaders handle small format differences):
#   FINANCIAL_CSV  -> needs: NAME (ZCTA/ZIP name or code), S1901_C01_012E (median HH income)
#   HEALTH_CSV     -> needs: ZCTA5 (5-digit), GHLTH_CrudePrev (general poor health %)
#   PHARMACY_CSV   -> needs: ZIP (5-digit), NAME (pharmacy name), X, Y (coords; optional)
#   POPULATION_CSV -> teammate used skiprows=10; script will autodetect 3 useful cols:
#                     ZCTA/ZIP code, population, and optionally land area (km2 or mi2)
#
# Outputs:
#   results/national_ifae_rank.csv
#   results/top5_ifae.csv
#
# Method:
#   - Build features by ZCTA/ZIP: median income, poor health %, population,
#     pharmacies per 10k population (as access proxy), and (if available) density
#   - Normalize features (0..1), form a transparent composite, and also fit
#     an IsolationForest on the feature vector. Final IFAE score = 0.5*composite +
#     0.5*iforest_anomaly (higher => more underserved / priority).
#
# Dependencies: pandas, numpy, scikit-learn (IsolationForest); pyarrow optional.
# =============================================================================

import os, time
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd

from sklearn.ensemble import IsolationForest

# === CONFIG: set your file paths here ===
FINANCIAL_CSV  = "data/financial_data.csv"     # needs: NAME, S1901_C01_012E
HEALTH_CSV     = "data/health_data.csv"        # needs: ZCTA5, GHLTH_CrudePrev
PHARMACY_CSV   = "data/pharmacy_data.csv"      # needs: ZIP, NAME, X, Y
POPULATION_CSV = "data/population_data.csv"    # your teammate used skiprows=10; 3 useful cols

OUT_DIR        = "results"                     # outputs: national_ifae_rank.csv, top5_ifae.csv
CONTAMINATION  = 0.03                          # IsolationForest sensitivity (try 0.02–0.05)
TOP_K          = 10

# ---------- Monitoring helpers ----------
_T0 = time.time()
def stamp(msg):
    now = datetime.now().strftime("%H:%M:%S")
    print(f"[{now} +{time.time()-_T0:6.2f}s] {msg}", flush=True)

class Step:
    def __init__(self, name): self.name=name; self.t0=None
    def __enter__(self): self.t0=time.time(); stamp(f"▶ {self.name} ..."); return self
    def __exit__(self, et, ev, tb):
        dt = time.time()-self.t0
        stamp(("✓ " if et is None else "✖ ") + f"{self.name} {'done' if et is None else 'failed'} in {dt:.2f}s")

def read_csv_smart(path, **kw):
    p = Path(path)
    if not p.exists(): raise FileNotFoundError(f"Missing file: {path}")
    try:
        return pd.read_csv(path, low_memory=False, **kw)
    except Exception as e:
        stamp(f"CSV read warning: {e}. Retrying with simpler defaults.")
        return pd.read_csv(path)

def minmax(s):
    x = pd.to_numeric(s, errors='coerce')
    mn, mx = x.min(skipna=True), x.max(skipna=True)
    if np.isnan(mn) or np.isnan(mx) or mx==mn:
        return pd.Series(0.0, index=x.index)
    return (x - mn) / (mx - mn)

def coerce_zcta(series):
    """Return 5-digit, zero-padded strings; strips non-digit chars if present."""
    s = series.astype(str).str.extract(r"(\d{3,5})", expand=False)
    return s.fillna("").str.zfill(5)

# ---------- Load & prepare each source ----------
with Step("Load FINANCIAL (income)"):
    fin = read_csv_smart(FINANCIAL_CSV)
    # Try to identify ZCTA code
    zcta_col = None
    for cand in ["ZCTA5","zcta5","ZCTA","zcta","ZIP","Zip","NAME","Name","name"]:
        if cand in fin.columns:
            zcta_col = cand; break
    if zcta_col is None:
        raise KeyError(f"Could not find a ZCTA/ZIP column in {FINANCIAL_CSV}. Columns: {list(fin.columns)[:10]} ...")
    if "S1901_C01_012E" not in fin.columns:
        raise KeyError("Missing S1901_C01_012E (median HH income) in FINANCIAL_CSV.")
    fin = fin.rename(columns={zcta_col:"ZCTA5"})
    fin["ZCTA5"] = coerce_zcta(fin["ZCTA5"])
    fin = fin[["ZCTA5","S1901_C01_012E"]].copy()
    fin["S1901_C01_012E"] = pd.to_numeric(fin["S1901_C01_012E"], errors="coerce")
    stamp(f"FIN rows={len(fin)}, null income={fin['S1901_C01_012E'].isna().sum()}")

with Step("Load HEALTH (poor general health %)"):
    hlth = read_csv_smart(HEALTH_CSV)
    if "ZCTA5" not in hlth.columns:
        # attempt alias
        for cand in ["ZCTA","zcta","ZIP","Zip","name","NAME"]:
            if cand in hlth.columns:
                hlth = hlth.rename(columns={cand:"ZCTA5"})
                break
    if "ZCTA5" not in hlth.columns or "GHLTH_CrudePrev" not in hlth.columns:
        raise KeyError("HEALTH_CSV must contain ZCTA5 and GHLTH_CrudePrev.")
    hlth["ZCTA5"] = coerce_zcta(hlth["ZCTA5"])
    hlth["GHLTH_CrudePrev"] = pd.to_numeric(hlth["GHLTH_CrudePrev"], errors="coerce")
    hlth = hlth[["ZCTA5","GHLTH_CrudePrev"]]
    stamp(f"HEALTH rows={len(hlth)}, null health%={hlth['GHLTH_CrudePrev'].isna().sum()}")

with Step("Load PHARMACY (access proxy)"):
    ph = read_csv_smart(PHARMACY_CSV)
    for needed in ["ZIP","NAME"]:
        if needed not in ph.columns:
            raise KeyError(f"PHARMACY_CSV must contain {needed}.")
    ph["ZCTA5"] = coerce_zcta(ph["ZIP"])
    # count unique pharmacies per ZCTA
    ph_cnt = ph.groupby("ZCTA5", dropna=False)["NAME"].nunique(dropna=True).reset_index()
    ph_cnt = ph_cnt.rename(columns={"NAME":"pharmacies_count"})
    stamp(f"PHARMACY unique ZCTA5={ph_cnt['ZCTA5'].nunique()}, total pharmacies={ph_cnt['pharmacies_count'].sum()}")

with Step("Load POPULATION (skiprows=10 heuristic + autodetect)"):
    pop = read_csv_smart(POPULATION_CSV, skiprows=10)
    # try to identify code + population + (optional) land area
    code_col = None
    for cand in ["ZCTA5","ZCTA","ZIP","Zip","GEOID","geoid","NAME","name"]:
        if cand in pop.columns:
            code_col=cand; break
    if code_col is None:
        # fallback: choose first object-like column
        obj_cols = [c for c in pop.columns if pop[c].dtype == object]
        code_col = obj_cols[0] if obj_cols else pop.columns[0]
    pop = pop.rename(columns={code_col:"ZCTA5"})
    pop["ZCTA5"] = coerce_zcta(pop["ZCTA5"])

    # population column guess
    pop_col = None
    for cand in ["POP","Population","population","TOTAL_POP","TotPop","DP05_0001E","pop"]:
        if cand in pop.columns:
            pop_col=cand; break
    if pop_col is None:
        # choose the largest-sum numeric column as population
        num_cols = [c for c in pop.columns if pd.api.types.is_numeric_dtype(pop[c])]
        sums = {c: pd.to_numeric(pop[c], errors="coerce").sum(skipna=True) for c in num_cols}
        pop_col = max(sums, key=sums.get) if sums else None
    if pop_col is None:
        raise KeyError("Could not infer population column in POPULATION_CSV.")

    # optional land area
    land_col = None
    for cand in ["land_area_km2","Land_Area_km2","ALAND_KM2","aland_km2","ALAND","area","AREA_KM2","ALAND_SQMI","AREALAND"]:
        if cand in pop.columns:
            land_col = cand; break

    keep_cols = ["ZCTA5", pop_col] + ([land_col] if land_col else [])
    pop = pop[keep_cols].copy()
    pop = pop.rename(columns={pop_col:"population"})
    pop["population"] = pd.to_numeric(pop["population"], errors="coerce")
    if land_col:
        pop = pop.rename(columns={land_col:"land_area_raw"})
        pop["land_area_raw"] = pd.to_numeric(pop["land_area_raw"], errors="coerce")

    stamp(f"POP rows={len(pop)}, null pop={pop['population'].isna().sum()}, land_col={land_col}")

# ---------- Merge to ZCTA frame ----------
with Step("Merge all features by ZCTA/ZIP"):
    df = fin.merge(hlth, on="ZCTA5", how="outer") \
            .merge(pop, on="ZCTA5", how="outer") \
            .merge(ph_cnt, on="ZCTA5", how="left")
    stamp(f"Merged shape: {df.shape}")
    print("Top null counts:\n", df.isna().sum().sort_values(ascending=False).tail(10).to_string(), flush=True)

# ---------- Feature engineering ----------
with Step("Feature engineering"):
    # income
    df["median_income"] = pd.to_numeric(df["S1901_C01_012E"], errors="coerce")
    # health burden
    df["poor_health_pct"] = pd.to_numeric(df["GHLTH_CrudePrev"], errors="coerce")
    # access proxy
    df["pharmacies_count"] = pd.to_numeric(df["pharmacies_count"], errors="coerce").fillna(0)
    # pharmacies per 10k (guard against div by zero)
    df["pharm_per_10k"] = np.where(df["population"].gt(0),
                                   1e4 * df["pharmacies_count"] / df["population"],
                                   np.nan)
    # population density if land provided; try to detect units:
    if "land_area_raw" in df.columns:
        # If raw area looks like square meters (ALAND), convert to km2
        # Heuristic: median ~10^6–10^9 for sq meters; <1e4 for km2
        med_area = np.nanmedian(df["land_area_raw"])
        if np.isfinite(med_area) and med_area > 1e5:
            df["land_area_km2"] = df["land_area_raw"] / 1e6
        else:
            df["land_area_km2"] = df["land_area_raw"]
        df["pop_density"] = np.where(df["land_area_km2"].gt(0),
                                     df["population"] / df["land_area_km2"],
                                     np.nan)
    else:
        df["pop_density"] = np.nan

    # Normalizations
    df["income_norm_inv"]   = 1.0 - minmax(df["median_income"])         # lower income -> higher need
    df["health_norm"]       = minmax(df["poor_health_pct"])             # higher poor-health% -> higher need
    df["access_norm_inv"]   = 1.0 - minmax(df["pharm_per_10k"])         # fewer pharmacies/10k -> higher need
    df["pop_norm"]          = minmax(df["population"])                  # prioritize where more people
    df["density_norm"]      = minmax(df["pop_density"])                 # optional

    for col in ["median_income","poor_health_pct","pharm_per_10k","population","pop_density"]:
        if col in df.columns:
            x = pd.to_numeric(df[col], errors="coerce")
            stamp(f"Range {col}: min={np.nanmin(x):.3f} max={np.nanmax(x):.3f} NaN={np.isnan(x).sum()}")

# ---------- Composite score ----------
with Step("Compute composite & train IsolationForest"):
    # Transparent composite (tweakable weights)
    w_income, w_health, w_access, w_pop, w_density = 0.30, 0.35, 0.25, 0.08, 0.02
    df["composite"] = (
        w_income  * df["income_norm_inv"].fillna(0) +
        w_health  * df["health_norm"].fillna(0) +
        w_access  * df["access_norm_inv"].fillna(0) +
        w_pop     * df["pop_norm"].fillna(0) +
        w_density * df["density_norm"].fillna(0)
    )

    # IsolationForest on normalized features
    feats = df[["income_norm_inv","health_norm","access_norm_inv","pop_norm","density_norm"]].fillna(0.0).values
    iforest = IsolationForest(random_state=42, contamination=CONTAMINATION, n_estimators=300)
    iforest.fit(feats)
    # decision_function: higher is less anomalous; we want higher = more underserved,
    # so invert and min-max
    decision = iforest.decision_function(feats)  # larger -> more normal
    inv = -decision
    # scale to 0..1
    inv = (inv - inv.min()) / (inv.max() - inv.min() + 1e-12)
    df["iforest_anomaly"] = inv

    # Final IFAE score = blend
    df["IFAE_score"] = 0.5 * df["composite"] + 0.5 * df["iforest_anomaly"]

with Step("Finalize outputs"):
    out = Path(OUT_DIR); out.mkdir(parents=True, exist_ok=True)

    # Prepare nice columns
    keep = [
        "ZCTA5","IFAE_score","composite","iforest_anomaly",
        "median_income","poor_health_pct","population","pharmacies_count","pharm_per_10k",
        "income_norm_inv","health_norm","access_norm_inv","pop_norm","density_norm","pop_density"
    ]
    keep = [c for c in keep if c in df.columns]
    ranked = df[keep].copy()
    ranked = ranked.sort_values("IFAE_score", ascending=False).reset_index(drop=True)

    # Write full national ranking & top-k
    national_path = out / "national_ifae_rank.csv"
    topk_path     = out / "top5_ifae.csv"
    ranked.to_csv(national_path, index=False)
    ranked.tail(TOP_K).to_csv(topk_path, index=False)

    stamp(f"Wrote {national_path}")
    stamp(f"Wrote {topk_path}")
    print("\nTop 5 preview:\n", ranked.head(TOP_K).to_string(index=False))

stamp("All done. 🚀")







[20:30:35 +  0.00s] ▶ Load FINANCIAL (income) ...
[20:30:35 +  0.29s] FIN rows=33773, null income=3264
[20:30:35 +  0.29s] ✓ Load FINANCIAL (income) done in 0.29s
[20:30:35 +  0.29s] ▶ Load HEALTH (poor general health %) ...
[20:30:35 +  0.48s] HEALTH rows=32520, null health%=0
[20:30:35 +  0.48s] ✓ Load HEALTH (poor general health %) done in 0.19s
[20:30:35 +  0.48s] ▶ Load PHARMACY (access proxy) ...
[20:30:36 +  0.94s] PHARMACY unique ZCTA5=14940, total pharmacies=61970
[20:30:36 +  0.94s] ✓ Load PHARMACY (access proxy) done in 0.47s
[20:30:36 +  0.94s] ▶ Load POPULATION (skiprows=10 heuristic + autodetect) ...
[20:30:36 +  0.99s] POP rows=40959, null pop=0, land_col=None
[20:30:36 +  0.99s] ✓ Load POPULATION (skiprows=10 heuristic + autodetect) done in 0.05s
[20:30:36 +  0.99s] ▶ Merge all features by ZCTA/ZIP ...
[20:30:36 +  1.03s] Merged shape: (41094, 5)
Top null counts:
 pharmacies_count    26171
S1901_C01_012E      10585
GHLTH_CrudePrev      8574
population            135
ZCT

/var/folders/9d/5gzqwh3d5sx43n9fsy97czyc0000gn/T/ipykernel_54177/2454373550.py:210: RuntimeWarning: All-NaN axis encountered
  stamp(f"Range {col}: min={np.nanmin(x):.3f} max={np.nanmax(x):.3f} NaN={np.isnan(x).sum()}")


[20:30:36 +  1.71s] ✓ Compute composite & train IsolationForest done in 0.68s
[20:30:36 +  1.71s] ▶ Finalize outputs ...
[20:30:37 +  1.89s] Wrote results/national_ifae_rank.csv
[20:30:37 +  1.89s] Wrote results/top5_ifae.csv

Top 5 preview:
 ZCTA5  IFAE_score  composite  iforest_anomaly  median_income  poor_health_pct  population  pharmacies_count  pharm_per_10k  income_norm_inv  health_norm  access_norm_inv  pop_norm  density_norm  pop_density
78521    0.851494   0.812017         0.890971        41276.0             40.7     92108.0              19.0       2.062796         0.856046     0.701323         0.998969  0.749980           0.0          NaN
90011    0.830166   0.786260         0.874073        53781.0             37.4    109414.0               4.0       0.365584         0.804682     0.638941         0.999817  0.890892           0.0          NaN
78207    0.817008   0.816558         0.817458        30655.0             43.1     54601.0              11.0       2.014615         0.899

In [ ]:
# ===================== CONFIG (edit paths as needed) =====================
FINANCIAL_CSV  = "data/financial_data.csv"     # expects: NAME, S1901_C01_012E
HEALTH_CSV     = "data/health_data.csv"        # expects: ZCTA5, GHLTH_CrudePrev
PHARMACY_CSV   = "data/pharmacy_data.csv"      # expects: ZIP, NAME, X, Y
POPULATION_CSV = "data/population_data.csv"    # teammate used skiprows=10 and cols [1,2,3]

OUT_DIR        = "results"
CONTAMINATION  = 0.03     # IsolationForest sensitivity (try 0.02–0.05)
TOP_K          = 5        # how many top/bottom rows to show/save
# =======================================================================

import os, warnings, numpy as np, pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA

# ---- Try TensorFlow (AE); if missing we'll use PCA fallback automatically ----
TF_AVAILABLE = True
try:
    import tensorflow as tf
    from tensorflow import keras
except Exception:
    TF_AVAILABLE = False
    warnings.warn("TensorFlow not found; using PCA reconstruction error instead of Autoencoder.")

# --------------------------- Utility functions ---------------------------

def winsorize(s: pd.Series, lo=0.01, hi=0.99) -> pd.Series:
    s = pd.to_numeric(s, errors='coerce')
    a, b = s.quantile(lo), s.quantile(hi)
    return s.clip(a, b)

def pct_rank(s: pd.Series) -> pd.Series:
    s = pd.to_numeric(s, errors="coerce")
    if s.nunique(dropna=True) <= 1:
        return pd.Series(0.5, index=s.index)
    return s.rank(pct=True, method='average')

def nonint_fraction(x: pd.Series) -> float:
    x = pd.to_numeric(x, errors='coerce')
    return float((np.abs(x - np.round(x)) > 1e-9).mean())

# ----------------------------- Readers (robust) -----------------------------

def read_financial_data(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    df = df[['NAME', 'S1901_C01_012E']].copy()
    df['zip'] = df['NAME'].astype(str).str.extract(r'(\d{5})')
    df['median_income'] = pd.to_numeric(df['S1901_C01_012E'], errors='coerce')
    return df[['zip','median_income']].dropna(subset=['zip'])

def read_health_data(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    df = df[['ZCTA5', 'GHLTH_CrudePrev']].copy()
    df['zip'] = df['ZCTA5'].astype(str).str.split('.').str[0].str.zfill(5)
    df['health_burden'] = pd.to_numeric(df['GHLTH_CrudePrev'], errors='coerce')
    return df[['zip','health_burden']]

def read_pharmacy_data(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    df = df[['ZIP', 'NAME', 'X', 'Y']].copy()
    df['zip'] = df['ZIP'].astype(str).str.zfill(5)
    counts = df.groupby('zip').size().reset_index(name='n_pharmacies')
    # Approx centroid from points (biased where pharmacies exist; OK as fallback)
    cent = df.groupby('zip')[['Y','X']].mean().rename(columns={'Y':'lat','X':'lon'}).reset_index()
    out = counts.merge(cent, on='zip', how='left')
    return out

def read_population_data(path: str) -> pd.DataFrame:
    """
    Tries to extract (zip, population, land_km2 OR pop_density).
    We replicate your teammate pattern (skiprows=10; select cols [1,2,3]) and infer.
    """
    df = pd.read_csv(path, skiprows=10)
    sub = df.iloc[:, [1,2,3]].copy()
    # detect ZCTA column
    zcol = None
    for c in sub.columns:
        if sub[c].astype(str).str.match(r'^\d{5}$').mean() > 0.4:
            zcol = c; break
    if zcol is None:
        for c in sub.columns:
            if sub[c].astype(str).str.contains(r'\d{5}').mean() > 0.4:
                zcol = c; break
    if zcol is None:
        zcol = sub.columns[0]
    rem = [c for c in sub.columns if c != zcol]
    a = pd.to_numeric(sub[rem[0]], errors='coerce')
    b = pd.to_numeric(sub[rem[1]], errors='coerce')

    # Heuristics:
    # - population: mostly integers, often > 100
    pop_like_a = ((a >= 50).mean() > 0.6) and ((np.abs(a - np.round(a)) < 1e-9).mean() > 0.5)
    pop_like_b = ((b >= 50).mean() > 0.6) and ((np.abs(b - np.round(b)) < 1e-9).mean() > 0.5)

    out = pd.DataFrame({'zip': sub[zcol].astype(str).str.extract(r'(\d{5})')[0].str.zfill(5)})

    population = None
    land_km2 = None
    pop_density = None

    if pop_like_a and not pop_like_b:
        population = a
        other = b
    elif pop_like_b and not pop_like_a:
        population = b
        other = a
    else:
        # neither looks like clean population; assume we're given a density-like column
        other = a if nonint_fraction(a) >= nonint_fraction(b) else b

    if population is not None:
        out['population'] = population.round().astype('Int64')
        # Try to infer land if the "other" looks huge (likely m^2) or reasonable (km^2)
        if other.notna().sum() > 0:
            med = other.median(skipna=True)
            if pd.notna(med):
                if med > 10000:        # probably m^2
                    land_km2 = other / 1e6
                else:
                    land_km2 = other    # assume already km^2 or density-ish small number
        if land_km2 is not None:
            out['land_km2'] = land_km2
            out['pop_density'] = out['population'] / out['land_km2'].replace(0, np.nan)
    else:
        # No population found; treat "other" as pop_density proxy
        pop_density = other
        out['pop_density'] = pop_density

    return out.dropna(subset=['zip'])

# ---------------------- Build master & engineered features ----------------------

def build_master(fin_path, hl_path, pharm_path, pop_path) -> pd.DataFrame:
    fin  = read_financial_data(fin_path)
    hlth = read_health_data(hl_path)
    pharm= read_pharmacy_data(pharm_path)
    pop  = read_population_data(pop_path)

    df = pharm.merge(fin, on='zip', how='outer') \
              .merge(hlth, on='zip', how='outer') \
              .merge(pop, on='zip', how='outer')

    # Fill pharmacy counts
    df['n_pharmacies'] = df['n_pharmacies'].fillna(0).astype(int)

    # Winsorize raw continuous fields before percentiling
    for c in ['median_income','health_burden','population','land_km2','pop_density']:
        if c in df.columns:
            df[c] = winsorize(df[c])

    # ---- Scarcity (preferred): people per pharmacy + pharmacies per 10k ----
    if 'population' in df.columns and df['population'].notna().any():
        denom = df['n_pharmacies'].replace(0, 1)              # keep finite when 0 pharmacies
        df['pop_per_pharmacy'] = df['population'] / denom
        df['pharm_per_10k']    = 10000 * df['n_pharmacies'] / df['population'].replace(0, np.nan)
        df['scarcity']         = df['pop_per_pharmacy']       # higher = worse
    else:
        # Fallback: density-based proxy (kept only if no population)
        warnings.warn("Population not found; using density-based scarcity proxy.")
        eps = 1e-9
        per_density = df['n_pharmacies'] / (pd.to_numeric(df['pop_density'], errors='coerce').fillna(0) + eps)
        df['scarcity'] = 1.0 / (1.0 + per_density)

    # ---- National percentiles ONLY (higher = worse) ----
    df['scarcity_pct']    = pct_rank(df['scarcity'])
    df['health_pct']      = pct_rank(df['health_burden'])
    df['fin_stress_pct']  = 1 - pct_rank(df['median_income'])
    if 'pop_density' in df.columns:
        df['dens_pct']    = pct_rank(df['pop_density'])
    else:
        df['dens_pct']    = 0.5  # neutral if unknown

    # ---- Transparent baseline (national percentiles) ----
    feat_national = ['scarcity_pct','health_pct','fin_stress_pct','dens_pct']
    df['baseline_score'] = df[feat_national].mean(axis=1)

    # ---- Model matrix uses the SAME national percentiles (no state-relative) ----
    Xdf = df[feat_national].fillna(0.5).astype(float)
    scaler = StandardScaler()
    X = scaler.fit_transform(Xdf.values)

    # ---- Isolation Forest ----
    iso = IsolationForest(n_estimators=400, contamination=CONTAMINATION, random_state=7, n_jobs=-1).fit(X)
    raw_if = -iso.score_samples(X)  # larger = more anomalous

    # ---- Autoencoder (or PCA fallback) ----
    if TF_AVAILABLE:
        tf.random.set_seed(7)
        D = X.shape[1]
        ae = keras.Sequential([
            keras.layers.Input(shape=(D,)),
            keras.layers.Dense(16, activation='relu', kernel_regularizer=keras.regularizers.l2(1e-4)),
            keras.layers.Dropout(0.05),
            keras.layers.Dense(8, activation='relu'),
            keras.layers.Dense(16, activation='relu'),
            keras.layers.Dense(D)  # linear head
        ])
        ae.compile(optimizer=keras.optimizers.Adam(1e-3), loss='mse')
        cb = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)]
        ae.fit(X, X, epochs=200, batch_size=256, validation_split=0.15, verbose=0, callbacks=cb)
        recon = ae.predict(X, verbose=0)
        raw_ae = ((X - recon)**2).mean(axis=1)
        ae_source = "autoencoder"
    else:
        pca = PCA(n_components=min(3, X.shape[1]-1 if X.shape[1]>1 else 1), random_state=7)
        Z = pca.fit_transform(X)
        recon = pca.inverse_transform(Z)
        raw_ae = ((X - recon)**2).mean(axis=1)
        ae_source = "pca"

    # ---- Ensemble anomaly percentiles ----
    df['if_pct']   = pct_rank(pd.Series(raw_if))
    df['ae_pct']   = pct_rank(pd.Series(raw_ae))
    df['anom_pct'] = 0.5*df['if_pct'] + 0.5*df['ae_pct']

    # ---- NO impact weighting (per request) ----
    # Priority = anomaly only
    df['priority'] = df['anom_pct']

    # ---- Final Rank = average rank of (priority) and (baseline) ----
    df['rank_priority'] = df['priority'].rank(ascending=False, method='average')
    df['rank_baseline'] = df['baseline_score'].rank(ascending=False, method='average')
    df['final_key']     = - (df['rank_priority'] + df['rank_baseline'])/2.0

    # ---- Order outputs ----
    cols = [
        'zip','n_pharmacies','median_income','health_burden',
    ]
    if 'population' in df.columns: cols += ['population']
    if 'land_km2'   in df.columns: cols += ['land_km2']
    if 'pop_density'in df.columns: cols += ['pop_density']

    cols += [
        'pop_per_pharmacy','pharm_per_10k','scarcity',
        'scarcity_pct','health_pct','fin_stress_pct','dens_pct',
        'baseline_score','if_pct','ae_pct','anom_pct','priority','final_key','lat','lon'
    ]
    cols = [c for c in cols if c in df.columns]  # keep only present

    out = df[cols].sort_values('final_key', ascending=True).reset_index(drop=True)

    meta = {
        "ae_source": ae_source,
        "used_state_relative": False,
        "used_impact_weighting": False
    }
    return out, meta

# ----------------------------- Run training & save -----------------------------

os.makedirs(OUT_DIR, exist_ok=True)
ranked, meta = build_master(FINANCIAL_CSV, HEALTH_CSV, PHARMACY_CSV, POPULATION_CSV)

full_path   = os.path.join(OUT_DIR, "national_ifae_rank.csv")
top_path    = os.path.join(OUT_DIR, "top5_ifae.csv")
bottom_path = os.path.join(OUT_DIR, "bottom5_ifae.csv")

ranked.to_csv(full_path, index=False)
ranked.head(TOP_K).to_csv(top_path, index=False)
ranked.tail(TOP_K).to_csv(bottom_path, index=False)

print(f"Second detector : {meta['ae_source']} (IF contamination={CONTAMINATION})")
print(f"State-relative percentiles used for modeling: {meta['used_state_relative']}")
print(f"Impact weighting used: {meta['used_impact_weighting']}")
print(f"Saved full rankings to  : {full_path}")
print(f"Saved top {TOP_K} to    : {top_path}")
print(f"Saved bottom {TOP_K} to : {bottom_path}")

# ------------------------------- Show results ---------------------------------
def _fmt(df):
    out = df.copy()
    for c in out.select_dtypes(include='number').columns:
        out[c] = out[c].astype(float).round(3)
    return out

print("\nTop candidates:")
show_cols = ['zip','priority','baseline_score','anom_pct','scarcity_pct','health_pct','fin_stress_pct','dens_pct','n_pharmacies']
if 'population' in ranked.columns: show_cols.append('population')
display(_fmt(ranked.head(TOP_K)[[c for c in show_cols if c in ranked.columns]]))

print("\nBottom candidates:")
display(_fmt(ranked.tail(TOP_K)[[c for c in show_cols if c in ranked.columns]]))


In [6]:
# =============================================================================
# IFAE (Income-Health-Access Equity) ZIP/ZCTA Ranking — with AQI + HHI
# - Transparent, tweakable composite (no state-relative, no impact weighting)
# - IsolationForest anomaly on the same normalized features
# - Integrates:
#     * AQI annual weighted average per ZIP (PM2.5): read_aqi_data()
#     * HHI Excel (heat vulnerability): read_hhi_excel()
# Outputs:
#   results/national_ifae_rank.csv
#   results/top5_ifae.csv
#   results/bottom5_ifae.csv
# =============================================================================

import os, time, warnings
from datetime import datetime
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest

# === CONFIG: set your file paths here ===
FINANCIAL_CSV  = "data/financial_data.csv"      # needs: NAME, S1901_C01_012E (median HH income)
HEALTH_CSV     = "data/health_data.csv"         # needs: ZCTA5, GHLTH_CrudePrev
PHARMACY_CSV   = "data/pharmacy_data.csv"       # needs: ZIP, NAME, X, Y
POPULATION_CSV = "data/population_data.csv"     # teammate used skiprows=10; autodetect columns

# NEW:
AQI_CSV        = "data/AQI_data.csv"            # PM2.5; arbitrary columns but should include ZIP, Observation Count, Arithmetic Mean, Month/Date Local
HHI_XLSX       = "data/HHI_data.xlsx"           # Excel; must include ZCTA and HHB_SCORE (optional: NBE_SCORE, OVERALL_SCORE)

OUT_DIR        = "results"
CONTAMINATION  = 0.03
TOP_K          = 10

# ---------- Monitoring helpers ----------
_T0 = time.time()
def stamp(msg):
    now = datetime.now().strftime("%H:%M:%S")
    print(f"[{now} +{time.time()-_T0:6.2f}s] {msg}", flush=True)

class Step:
    def __init__(self, name): self.name=name; self.t0=None
    def __enter__(self): self.t0=time.time(); stamp(f"▶ {self.name} ..."); return self
    def __exit__(self, et, ev, tb):
        dt = time.time()-self.t0
        stamp(("✓ " if et is None else "✖ ") + f"{self.name} {'done' if et is None else 'failed'} in {dt:.2f}s")

def read_csv_smart(path, **kw):
    p = Path(path)
    if not p.exists(): raise FileNotFoundError(f"Missing file: {path}")
    try:
        return pd.read_csv(path, low_memory=False, **kw)
    except Exception as e:
        stamp(f"CSV read warning: {e}. Retrying with simpler defaults.")
        return pd.read_csv(path)

def minmax(s):
    x = pd.to_numeric(s, errors='coerce')
    mn, mx = x.min(skipna=True), x.max(skipna=True)
    if np.isnan(mn) or np.isnan(mx) or mx==mn:
        return pd.Series(0.0, index=x.index)
    return (x - mn) / (mx - mn)

def coerce_zcta(series):
    """Return 5-digit, zero-padded strings; strips non-digit chars if present."""
    s = series.astype(str).str.extract(r"(\d{3,5})", expand=False)
    return s.fillna("").str.zfill(5)

def nonint_fraction(x: pd.Series) -> float:
    x = pd.to_numeric(x, errors='coerce')
    return float((np.abs(x - np.round(x)) > 1e-9).mean())

# ---------- New loaders: AQI + HHI ----------
def read_aqi_data(file_path):
    """
    Reads AQI/PM2.5-like CSV and aggregates to:
      - annual weighted average per ZIP (by Observation Count)
    Returns: aqi_annual: [zip, aqi, obs_total]
    """
    df = read_csv_smart(file_path)
    # choose ZIP column (some files have ZIP and ZIP.1)
    zip_cols = [c for c in df.columns if str(c).upper().startswith('ZIP')]
    if not zip_cols:
        raise KeyError("AQI CSV needs a ZIP column.")
    zip_col = zip_cols[-1]
    df['zip'] = df[zip_col].astype(str).str.extract(r'(\d{5})')[0].fillna('').str.zfill(5)

    if 'Arithmetic Mean' not in df.columns or 'Observation Count' not in df.columns:
        raise KeyError("AQI CSV must include 'Arithmetic Mean' and 'Observation Count' columns.")
    val_col = 'Arithmetic Mean'
    w_col   = 'Observation Count'
    df[val_col] = pd.to_numeric(df[val_col], errors='coerce')
    df[w_col]   = pd.to_numeric(df[w_col], errors='coerce').fillna(0)

    # Restrict to PM2.5 if present
    if 'Parameter Name' in df.columns:
        df = df[df['Parameter Name'].astype(str).str.contains('PM2.5', na=False)]

    # Drop junk
    df = df.dropna(subset=['zip', val_col])
    df = df[df[w_col] > 0]

    # Annual weighted average per ZIP
    grp_annual = df.groupby('zip', as_index=False).apply(
        lambda g: pd.Series({
            'aqi': np.average(g[val_col], weights=g[w_col]),
            'obs_total': g[w_col].sum()
        })
    ).reset_index(drop=True)

    return grp_annual[['zip','aqi','obs_total']]

def read_hhi_excel(file_path):
    """
    Reads Heat-Health Index (HHI) Excel and returns:
      [zip, heat_hhb, nbe_score?, hhi_overall?]
    """
    df = pd.read_excel(file_path, dtype={'ZCTA': str})
    if 'ZCTA' not in df.columns:
        raise ValueError("HHI Excel must contain 'ZCTA' column.")
    df['zip'] = df['ZCTA'].astype(str).str.extract(r'(\d{5})')[0].fillna('').str.zfill(5)

    out = pd.DataFrame({'zip': df['zip']})
    if 'HHB_SCORE' in df.columns:
        out['heat_hhb'] = pd.to_numeric(df['HHB_SCORE'], errors='coerce')
    if 'NBE_SCORE' in df.columns:
        out['nbe_score'] = pd.to_numeric(df['NBE_SCORE'], errors='coerce')
    if 'OVERALL_SCORE' in df.columns:
        out['hhi_overall'] = pd.to_numeric(df['OVERALL_SCORE'], errors='coerce')

    return out.dropna(subset=['zip']).drop_duplicates(subset=['zip'])

# ---------- Load & prepare each source ----------
with Step("Load FINANCIAL (income)"):
    fin = read_csv_smart(FINANCIAL_CSV)
    zcta_col = None
    for cand in ["ZCTA5","zcta5","ZCTA","zcta","ZIP","Zip","NAME","Name","name"]:
        if cand in fin.columns:
            zcta_col = cand; break
    if zcta_col is None:
        raise KeyError(f"Could not find a ZCTA/ZIP column in {FINANCIAL_CSV}. Columns: {list(fin.columns)[:10]} ...")
    if "S1901_C01_012E" not in fin.columns:
        raise KeyError("Missing S1901_C01_012E (median HH income) in FINANCIAL_CSV.")
    fin = fin.rename(columns={zcta_col:"ZCTA5"})
    fin["ZCTA5"] = coerce_zcta(fin["ZCTA5"])
    fin = fin[["ZCTA5","S1901_C01_012E"]].copy()
    fin["S1901_C01_012E"] = pd.to_numeric(fin["S1901_C01_012E"], errors="coerce")
    stamp(f"FIN rows={len(fin)}, null income={fin['S1901_C01_012E'].isna().sum()}")

with Step("Load HEALTH (poor general health %)"):
    hlth = read_csv_smart(HEALTH_CSV)
    if "ZCTA5" not in hlth.columns:
        for cand in ["ZCTA","zcta","ZIP","Zip","name","NAME"]:
            if cand in hlth.columns:
                hlth = hlth.rename(columns={cand:"ZCTA5"})
                break
    if "ZCTA5" not in hlth.columns or "GHLTH_CrudePrev" not in hlth.columns:
        raise KeyError("HEALTH_CSV must contain ZCTA5 and GHLTH_CrudePrev.")
    hlth["ZCTA5"] = coerce_zcta(hlth["ZCTA5"])
    hlth["GHLTH_CrudePrev"] = pd.to_numeric(hlth["GHLTH_CrudePrev"], errors="coerce")
    hlth = hlth[["ZCTA5","GHLTH_CrudePrev"]]
    stamp(f"HEALTH rows={len(hlth)}, null health%={hlth['GHLTH_CrudePrev'].isna().sum()}")

with Step("Load PHARMACY (access proxy)"):
    ph = read_csv_smart(PHARMACY_CSV)
    for needed in ["ZIP","NAME"]:
        if needed not in ph.columns:
            raise KeyError(f"PHARMACY_CSV must contain {needed}.")
    ph["ZCTA5"] = coerce_zcta(ph["ZIP"])
    ph_cnt = ph.groupby("ZCTA5", dropna=False)["NAME"].nunique(dropna=True).reset_index()
    ph_cnt = ph_cnt.rename(columns={"NAME":"pharmacies_count"})
    stamp(f"PHARMACY unique ZCTA5={ph_cnt['ZCTA5'].nunique()}, total pharmacies={ph_cnt['pharmacies_count'].sum()}")

with Step("Load POPULATION (skiprows=10 heuristic + autodetect)"):
    pop = read_csv_smart(POPULATION_CSV, skiprows=10)
    code_col = None
    for cand in ["ZCTA5","ZCTA","ZIP","Zip","GEOID","geoid","NAME","name"]:
        if cand in pop.columns:
            code_col=cand; break
    if code_col is None:
        obj_cols = [c for c in pop.columns if pop[c].dtype == object]
        code_col = obj_cols[0] if obj_cols else pop.columns[0]
    pop = pop.rename(columns={code_col:"ZCTA5"})
    pop["ZCTA5"] = coerce_zcta(pop["ZCTA5"])

    pop_col = None
    for cand in ["POP","Population","population","TOTAL_POP","TotPop","DP05_0001E","pop"]:
        if cand in pop.columns:
            pop_col=cand; break
    if pop_col is None:
        num_cols = [c for c in pop.columns if pd.api.types.is_numeric_dtype(pop[c])]
        sums = {c: pd.to_numeric(pop[c], errors="coerce").sum(skipna=True) for c in num_cols}
        pop_col = max(sums, key=sums.get) if sums else None
    if pop_col is None:
        raise KeyError("Could not infer population column in POPULATION_CSV.")

    land_col = None
    for cand in ["land_area_km2","Land_Area_km2","ALAND_KM2","aland_km2","ALAND","area","AREA_KM2","ALAND_SQMI","AREALAND"]:
        if cand in pop.columns:
            land_col = cand; break

    keep_cols = ["ZCTA5", pop_col] + ([land_col] if land_col else [])
    pop = pop[keep_cols].copy()
    pop = pop.rename(columns={pop_col:"population"})
    pop["population"] = pd.to_numeric(pop["population"], errors="coerce")
    if land_col:
        pop = pop.rename(columns={land_col:"land_area_raw"})
        pop["land_area_raw"] = pd.to_numeric(pop["land_area_raw"], errors="coerce")
    stamp(f"POP rows={len(pop)}, null pop={pop['population'].isna().sum()}, land_col={land_col}")

# NEW: load AQI + HHI (optional; skip if files missing)
try:
    with Step("Load AQI (annual weighted PM2.5)"):
        aqi_annual = read_aqi_data(AQI_CSV)
        stamp(f"AQI rows={len(aqi_annual)}, example:\n{aqi_annual.head(3).to_string(index=False)}")
except Exception as e:
    aqi_annual = None
    warnings.warn(f"AQI not loaded: {e}")

try:
    with Step("Load HHI (heat vulnerability)"):
        hhi = read_hhi_excel(HHI_XLSX)
        stamp(f"HHI rows={len(hhi)}, example:\n{hhi.head(3).to_string(index=False)}")
except Exception as e:
    hhi = None
    warnings.warn(f"HHI not loaded: {e}")

# ---------- Merge all to ZCTA frame ----------
with Step("Merge all features by ZCTA/ZIP"):
    df = fin.merge(hlth, on="ZCTA5", how="outer") \
            .merge(pop, on="ZCTA5", how="outer") \
            .merge(ph_cnt, on="ZCTA5", how="left")
    if aqi_annual is not None and not aqi_annual.empty:
        df = df.merge(aqi_annual.rename(columns={'zip':'ZCTA5'}), on="ZCTA5", how="left")
    if hhi is not None and not hhi.empty:
        df = df.merge(hhi.rename(columns={'zip':'ZCTA5'}), on="ZCTA5", how="left")
    stamp(f"Merged shape: {df.shape}")
    print("Null counts (top):\n", df.isna().sum().sort_values(ascending=False).head(10).to_string(), flush=True)

# ---------- Feature engineering ----------
with Step("Feature engineering"):
    # income
    df["median_income"] = pd.to_numeric(df["S1901_C01_012E"], errors="coerce")
    # health burden
    df["poor_health_pct"] = pd.to_numeric(df["GHLTH_CrudePrev"], errors="coerce")
    # pharmacies
    df["pharmacies_count"] = pd.to_numeric(df["pharmacies_count"], errors="coerce").fillna(0)

    # pharmacies per 10k (guard against div by zero)
    df["pharm_per_10k"] = np.where(df["population"].gt(0),
                                   1e4 * df["pharmacies_count"] / df["population"],
                                   np.nan)

    # population density if land provided; convert sq meters to km2 if needed
    if "land_area_raw" in df.columns:
        med_area = np.nanmedian(df["land_area_raw"])
        if np.isfinite(med_area) and med_area > 1e5:
            df["land_area_km2"] = df["land_area_raw"] / 1e6
        else:
            df["land_area_km2"] = df["land_area_raw"]
        df["pop_density"] = np.where(df["land_area_km2"].gt(0),
                                     df["population"] / df["land_area_km2"],
                                     np.nan)
    else:
        df["pop_density"] = np.nan

    # NEW: AQI (higher = worse); HHI heat_hhb (higher = worse)
    if 'aqi' in df.columns:
        df['aqi'] = pd.to_numeric(df['aqi'], errors='coerce')
    if 'heat_hhb' in df.columns:
        df['heat_hhb'] = pd.to_numeric(df['heat_hhb'], errors='coerce')

    # Normalizations (0..1, higher = worse)
    df["income_norm_inv"]   = 1.0 - minmax(df["median_income"])
    df["health_norm"]       = minmax(df["poor_health_pct"])
    df["access_norm_inv"]   = 1.0 - minmax(df["pharm_per_10k"])
    df["pop_norm"]          = minmax(df["population"])     # used only in reporting; NOT a rank weight
    df["density_norm"]      = minmax(df["pop_density"])
    df["aqi_norm"]          = minmax(df["aqi"]) if 'aqi' in df.columns else 0.0
    df["heat_norm"]         = minmax(df["heat_hhb"]) if 'heat_hhb' in df.columns else 0.0

    for col in ["median_income","poor_health_pct","pharm_per_10k","population","pop_density","aqi","heat_hhb"]:
        if col in df.columns:
            x = pd.to_numeric(df[col], errors="coerce")
            stamp(f"Range {col}: min={np.nanmin(x):.3f} max={np.nanmax(x):.3f} NaN={np.isnan(x).sum()}")

# ---------- Composite + IsolationForest ----------
with Step("Compute composite & train IsolationForest"):
    # Transparent composite (tweakable weights; sum to 1)
    # Emphasize: access, health, income; include AQI & heat if present; tiny weight for density.
    has_aqi  = ('aqi_norm'  in df.columns) and (df['aqi_norm'].dtype != object)
    has_heat = ('heat_norm' in df.columns) and (df['heat_norm'].dtype != object)

    if has_aqi and has_heat:
        w_income, w_health, w_access, w_aqi, w_heat, w_density = 0.25, 0.28, 0.30, 0.08, 0.07, 0.02
    elif has_aqi and not has_heat:
        w_income, w_health, w_access, w_aqi, w_heat, w_density = 0.27, 0.30, 0.33, 0.08, 0.00, 0.02
    elif has_heat and not has_aqi:
        w_income, w_health, w_access, w_aqi, w_heat, w_density = 0.27, 0.30, 0.33, 0.00, 0.08, 0.02
    else:
        w_income, w_health, w_access, w_aqi, w_heat, w_density = 0.30, 0.35, 0.33, 0.00, 0.00, 0.02

    df["composite"] = (
        w_income  * df["income_norm_inv"].fillna(0) +
        w_health  * df["health_norm"].fillna(0) +
        w_access  * df["access_norm_inv"].fillna(0) +
        w_aqi     * (df["aqi_norm"].fillna(0)   if has_aqi  else 0) +
        w_heat    * (df["heat_norm"].fillna(0)  if has_heat else 0) +
        w_density * df["density_norm"].fillna(0)
    )

    # IF features = the same normalized vector as composite (no state-relative, no impact weighting)
    feat_cols = ["income_norm_inv","health_norm","access_norm_inv","density_norm"]
    if has_aqi:  feat_cols.append("aqi_norm")
    if has_heat: feat_cols.append("heat_norm")
    feats = df[feat_cols].fillna(0.0).values

    iforest = IsolationForest(random_state=42, contamination=CONTAMINATION, n_estimators=200, n_jobs=-1)
    iforest.fit(feats)
    decision = iforest.decision_function(feats)  # higher -> more "normal"
    inv = -decision
    inv = (inv - inv.min()) / (inv.max() - inv.min() + 1e-12)  # 0..1
    df["iforest_anomaly"] = inv

    # Final IFAE score = 0.5*composite + 0.5*iforest_anomaly (no population/density impact multiplier)
    df["IFAE_score"] = 0.5 * df["composite"] + 0.5 * df["iforest_anomaly"]

with Step("Finalize outputs"):
    out = Path(OUT_DIR); out.mkdir(parents=True, exist_ok=True)

    keep = [
        "ZCTA5","IFAE_score","composite","iforest_anomaly",
        "median_income","poor_health_pct","population","pharmacies_count","pharm_per_10k",
        "income_norm_inv","health_norm","access_norm_inv","density_norm","pop_density","pop_norm"
    ]
    if 'aqi' in df.columns:
        keep += ['aqi','aqi_norm','obs_total']
    if 'heat_hhb' in df.columns:
        keep += ['heat_hhb','heat_norm']
    if 'nbe_score' in df.columns:     keep += ['nbe_score']
    if 'hhi_overall' in df.columns:   keep += ['hhi_overall']

    keep = [c for c in keep if c in df.columns]
    ranked = df[keep].copy().sort_values("IFAE_score", ascending=False).reset_index(drop=True)

    national_path = out / "national_ifae_rank.csv"
    topk_path     = out / "top5_ifae.csv"
    bottomk_path  = out / "bottom5_ifae.csv"

    ranked.to_csv(national_path, index=False)
    ranked.head(TOP_K).to_csv(topk_path, index=False)      # <-- Top K (best)
    ranked.tail(TOP_K).to_csv(bottomk_path, index=False)   # <-- Bottom K

    stamp(f"Wrote {national_path}")
    stamp(f"Wrote {topk_path}")
    stamp(f"Wrote {bottomk_path}")
    print("\nTop 5 preview:\n", ranked.head(5).to_string(index=False))
    print("\nBottom 5 preview:\n", ranked.tail(5).to_string(index=False))

stamp("All done. 🚀")


[20:52:11 +  0.00s] ▶ Load FINANCIAL (income) ...
[20:52:11 +  0.33s] FIN rows=33773, null income=3264
[20:52:11 +  0.33s] ✓ Load FINANCIAL (income) done in 0.33s
[20:52:11 +  0.33s] ▶ Load HEALTH (poor general health %) ...
[20:52:11 +  0.55s] HEALTH rows=32520, null health%=0
[20:52:11 +  0.55s] ✓ Load HEALTH (poor general health %) done in 0.22s
[20:52:11 +  0.55s] ▶ Load PHARMACY (access proxy) ...
[20:52:12 +  1.24s] PHARMACY unique ZCTA5=14940, total pharmacies=61970
[20:52:12 +  1.24s] ✓ Load PHARMACY (access proxy) done in 0.70s
[20:52:12 +  1.24s] ▶ Load POPULATION (skiprows=10 heuristic + autodetect) ...
[20:52:12 +  1.30s] POP rows=40959, null pop=0, land_col=None
[20:52:12 +  1.30s] ✓ Load POPULATION (skiprows=10 heuristic + autodetect) done in 0.05s
[20:52:12 +  1.30s] ▶ Load AQI (annual weighted PM2.5) ...
[20:52:14 +  2.84s] AQI rows=125, example:
  zip      aqi  obs_total
00000 7.016117  7489991.0
10065 2.595833       72.0
10303 6.532000      150.0
[20:52:14 +  2.85s] ✓

/var/folders/9d/5gzqwh3d5sx43n9fsy97czyc0000gn/T/ipykernel_54177/1835354442.py:104: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grp_annual = df.groupby('zip', as_index=False).apply(
/var/folders/9d/5gzqwh3d5sx43n9fsy97czyc0000gn/T/ipykernel_54177/1835354442.py:226: UserWarning: HHI not loaded: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.
  warnings.warn(f"HHI not loaded: {e}")
/var/folders/9d/5gzqwh3d5sx43n9fsy97czyc0000gn/T/ipykernel_54177/1835354442.py:285: RuntimeWarning: All-NaN axis encountered
  stamp(f"Range {col}: min={np.nanmin(x):.3f} max={np.nanmax(x):.3f} NaN={np.isnan(x).sum()}")


[20:52:14 +  3.36s] ✓ Compute composite & train IsolationForest done in 0.47s
[20:52:14 +  3.37s] ▶ Finalize outputs ...
[20:52:14 +  3.56s] Wrote results/national_ifae_rank.csv
[20:52:14 +  3.56s] Wrote results/top5_ifae.csv
[20:52:14 +  3.56s] Wrote results/bottom5_ifae.csv

Top 5 preview:
 ZCTA5  IFAE_score  composite  iforest_anomaly  median_income  poor_health_pct  population  pharmacies_count  pharm_per_10k  income_norm_inv  health_norm  access_norm_inv  density_norm  pop_density  pop_norm   aqi  aqi_norm  obs_total
06702    0.791762   0.769051         0.814474        16198.0             47.2      3025.0               3.0       9.917355         0.959053     0.824197         0.995041           0.0          NaN  0.024631   NaN       NaN        NaN
44702    0.783916   0.712806         0.855025        13159.0             36.6       948.0               3.0      31.645570         0.971535     0.623819         0.984177           0.0          NaN  0.007719   NaN       NaN        NaN
4164

In [9]:
# =============================================================================
# IFAE (Income-Health-Access Equity) ZIP/ZCTA Ranking — with AQI + HHI
# - Composite (percentiles) + IsolationForest anomaly
# - Population-based access (people per pharmacy)
# - Auto-disable low-coverage features; impute income/health by national medians
# - No state-relative, no population/density impact weighting
#
# Outputs:
#   results/national_ifae_rank.csv   (full ranking)
#   results/top5_ifae.csv            (top K with population >= 1,000)
#   results/bottom5_ifae.csv         (bottom K)
# =============================================================================

import os, time, warnings
from datetime import datetime
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest

# === CONFIG: set your file paths here ===
FINANCIAL_CSV  = "data/financial_data.csv"      # needs: NAME, S1901_C01_012E (median HH income)
HEALTH_CSV     = "data/health_data.csv"         # needs: ZCTA5, GHLTH_CrudePrev
PHARMACY_CSV   = "data/pharmacy_data.csv"       # needs: ZIP, NAME, X, Y (X/Y optional)
POPULATION_CSV = "data/population_data.csv"     # teammate used skiprows=10; autodetect columns

# Optional extras (will be auto-disabled if missing/low coverage)
AQI_CSV        = "data/AQI_data.csv"            # PM2.5; must include ZIP, Arithmetic Mean, Observation Count
HHI_XLSX       = "data/HHI_data.xlsx"           # Excel; must include ZCTA, HHB_SCORE (heat)

OUT_DIR        = "results"
CONTAMINATION  = 0.03
TOP_K          = 10
MIN_POP_TOPK   = 1000   # population gate for top-k presentation

# ---------- Monitoring helpers ----------
_T0 = time.time()
def stamp(msg):
    now = datetime.now().strftime("%H:%M:%S")
    print(f"[{now} +{time.time()-_T0:6.2f}s] {msg}", flush=True)

class Step:
    def __init__(self, name): self.name=name; self.t0=None
    def __enter__(self): self.t0=time.time(); stamp(f"▶ {self.name} ..."); return self
    def __exit__(self, et, ev, tb):
        dt = time.time()-self.t0
        stamp(("✓ " if et is None else "✖ ") + f"{self.name} {'done' if et is None else 'failed'} in {dt:.2f}s")

def read_csv_smart(path, **kw):
    p = Path(path)
    if not p.exists(): raise FileNotFoundError(f"Missing file: {path}")
    try:
        return pd.read_csv(path, low_memory=False, **kw)
    except Exception as e:
        stamp(f"CSV read warning: {e}. Retrying with simpler defaults.")
        return pd.read_csv(path)

def coerce_zcta(series):
    """Return 5-digit, zero-padded strings; strips non-digit chars if present."""
    s = series.astype(str).str.extract(r"(\d{3,5})", expand=False)
    return s.fillna("").str.zfill(5)

def pct_rank(s: pd.Series) -> pd.Series:
    s = pd.to_numeric(s, errors='coerce')
    return s.rank(pct=True, method='average').fillna(0.5)

def nonint_fraction(x: pd.Series) -> float:
    x = pd.to_numeric(x, errors='coerce')
    return float((np.abs(x - np.round(x)) > 1e-9).mean())

# ---------- Optional loaders: AQI + HHI ----------
def read_aqi_data(file_path):
    """
    Reads AQI/PM2.5-like CSV and aggregates annual weighted average per ZIP by Observation Count.
    Returns: aqi_annual: [zip, aqi, obs_total]
    """
    df = read_csv_smart(file_path)
    zip_cols = [c for c in df.columns if str(c).upper().startswith('ZIP')]
    if not zip_cols:
        raise KeyError("AQI CSV needs a ZIP column.")
    zip_col = zip_cols[-1]
    df['zip'] = df[zip_col].astype(str).str.extract(r'(\d{5})')[0].fillna('').str.zfill(5)

    if 'Arithmetic Mean' not in df.columns or 'Observation Count' not in df.columns:
        raise KeyError("AQI CSV must include 'Arithmetic Mean' and 'Observation Count' columns.")
    val_col = 'Arithmetic Mean'
    w_col   = 'Observation Count'
    df[val_col] = pd.to_numeric(df[val_col], errors='coerce')
    df[w_col]   = pd.to_numeric(df[w_col], errors='coerce').fillna(0)

    # Restrict to PM2.5 if present
    if 'Parameter Name' in df.columns:
        df = df[df['Parameter Name'].astype(str).str.contains('PM2.5', na=False)]

    df = df.dropna(subset=['zip', val_col])
    df = df[df[w_col] > 0]

    grp_annual = df.groupby('zip', as_index=False).apply(
        lambda g: pd.Series({
            'aqi': np.average(g[val_col], weights=g[w_col]),
            'obs_total': g[w_col].sum()
        })
    ).reset_index(drop=True)

    return grp_annual[['zip','aqi','obs_total']]

def read_hhi_excel(file_path):
    """Reads Heat-Health Index Excel -> [zip, heat_hhb, nbe_score?, hhi_overall?]"""
    df = pd.read_excel(file_path, dtype={'ZCTA': str})
    if 'ZCTA' not in df.columns:
        raise ValueError("HHI Excel must contain 'ZCTA' column.")
    df['zip'] = df['ZCTA'].astype(str).str.extract(r'(\d{5})')[0].fillna('').str.zfill(5)

    out = pd.DataFrame({'zip': df['zip']})
    if 'HHB_SCORE' in df.columns:
        out['heat_hhb'] = pd.to_numeric(df['HHB_SCORE'], errors='coerce')
    if 'NBE_SCORE' in df.columns:
        out['nbe_score'] = pd.to_numeric(df['NBE_SCORE'], errors='coerce')
    if 'OVERALL_SCORE' in df.columns:
        out['hhi_overall'] = pd.to_numeric(df['OVERALL_SCORE'], errors='coerce')

    return out.dropna(subset=['zip']).drop_duplicates(subset=['zip'])

# ---------- Load & prepare each source ----------
with Step("Load FINANCIAL (income)"):
    fin = read_csv_smart(FINANCIAL_CSV)
    zcta_col = None
    for cand in ["ZCTA5","zcta5","ZCTA","zcta","ZIP","Zip","NAME","Name","name"]:
        if cand in fin.columns:
            zcta_col = cand; break
    if zcta_col is None:
        raise KeyError(f"Could not find a ZCTA/ZIP column in {FINANCIAL_CSV}.")
    if "S1901_C01_012E" not in fin.columns:
        raise KeyError("Missing S1901_C01_012E (median HH income) in FINANCIAL_CSV.")
    fin = fin.rename(columns={zcta_col:"ZCTA5"})
    fin["ZCTA5"] = coerce_zcta(fin["ZCTA5"])
    fin = fin[["ZCTA5","S1901_C01_012E"]].copy()
    fin["S1901_C01_012E"] = pd.to_numeric(fin["S1901_C01_012E"], errors="coerce")
    stamp(f"FIN rows={len(fin)}, null income={fin['S1901_C01_012E'].isna().sum()}")

with Step("Load HEALTH (poor general health %)"):
    hlth = read_csv_smart(HEALTH_CSV)
    if "ZCTA5" not in hlth.columns:
        for cand in ["ZCTA","zcta","ZIP","Zip","name","NAME"]:
            if cand in hlth.columns:
                hlth = hlth.rename(columns={cand:"ZCTA5"})
                break
    if "ZCTA5" not in hlth.columns or "GHLTH_CrudePrev" not in hlth.columns:
        raise KeyError("HEALTH_CSV must contain ZCTA5 and GHLTH_CrudePrev.")
    hlth["ZCTA5"] = coerce_zcta(hlth["ZCTA5"])
    hlth["GHLTH_CrudePrev"] = pd.to_numeric(hlth["GHLTH_CrudePrev"], errors="coerce")
    hlth = hlth[["ZCTA5","GHLTH_CrudePrev"]]
    stamp(f"HEALTH rows={len(hlth)}, null health%={hlth['GHLTH_CrudePrev'].isna().sum()}")

with Step("Load PHARMACY (access)"):
    ph = read_csv_smart(PHARMACY_CSV)
    for needed in ["ZIP","NAME"]:
        if needed not in ph.columns:
            raise KeyError(f"PHARMACY_CSV must contain {needed}.")
    ph["ZCTA5"] = coerce_zcta(ph["ZIP"])
    ph_cnt = ph.groupby("ZCTA5", dropna=False)["NAME"].nunique(dropna=True).reset_index()
    ph_cnt = ph_cnt.rename(columns={"NAME":"pharmacies_count"})
    stamp(f"PHARMACY unique ZCTA5={ph_cnt['ZCTA5'].nunique()}, pharmacies={ph_cnt['pharmacies_count'].sum()}")

with Step("Load POPULATION (skiprows=10 + autodetect)"):
    pop = read_csv_smart(POPULATION_CSV, skiprows=10)
    code_col = None
    for cand in ["ZCTA5","ZCTA","ZIP","Zip","GEOID","geoid","NAME","name"]:
        if cand in pop.columns:
            code_col=cand; break
    if code_col is None:
        obj_cols = [c for c in pop.columns if pop[c].dtype == object]
        code_col = obj_cols[0] if obj_cols else pop.columns[0]
    pop = pop.rename(columns={code_col:"ZCTA5"})
    pop["ZCTA5"] = coerce_zcta(pop["ZCTA5"])

    pop_col = None
    for cand in ["POP","Population","population","TOTAL_POP","TotPop","DP05_0001E","pop"]:
        if cand in pop.columns:
            pop_col=cand; break
    if pop_col is None:
        num_cols = [c for c in pop.columns if pd.api.types.is_numeric_dtype(pop[c])]
        sums = {c: pd.to_numeric(pop[c], errors="coerce").sum(skipna=True) for c in num_cols}
        pop_col = max(sums, key=sums.get) if sums else None
    if pop_col is None:
        raise KeyError("Could not infer population column in POPULATION_CSV.")

    land_col = None
    for cand in ["land_area_km2","Land_Area_km2","ALAND_KM2","aland_km2","ALAND","area","AREA_KM2","ALAND_SQMI","AREALAND"]:
        if cand in pop.columns:
            land_col = cand; break

    keep_cols = ["ZCTA5", pop_col] + ([land_col] if land_col else [])
    pop = pop[keep_cols].copy()
    pop = pop.rename(columns={pop_col:"population"})
    pop["population"] = pd.to_numeric(pop["population"], errors="coerce")
    if land_col:
        pop = pop.rename(columns={land_col:"land_area_raw"})
        pop["land_area_raw"] = pd.to_numeric(pop["land_area_raw"], errors="coerce")
    stamp(f"POP rows={len(pop)}, null pop={pop['population'].isna().sum()}, land_col={land_col}")

# Optional: load AQI + HHI
try:
    with Step("Load AQI (annual weighted PM2.5)"):
        aqi_annual = read_aqi_data(AQI_CSV)
        stamp(f"AQI rows={len(aqi_annual)} (coverage ~{100*len(aqi_annual)/max(1, pop['ZCTA5'].nunique()):.1f}%)")
except Exception as e:
    aqi_annual = None
    warnings.warn(f"AQI not loaded: {e}")

try:
    with Step("Load HHI (heat vulnerability)"):
        hhi = read_hhi_excel(HHI_XLSX)
        stamp(f"HHI rows={len(hhi)}")
except Exception as e:
    hhi = None
    warnings.warn(f"HHI not loaded: {e}")

# ---------- Merge all to ZCTA frame ----------
with Step("Merge all features by ZCTA/ZIP"):
    df = fin.merge(hlth, on="ZCTA5", how="outer") \
            .merge(pop, on="ZCTA5", how="outer") \
            .merge(ph_cnt, on="ZCTA5", how="left")
    if aqi_annual is not None and not aqi_annual.empty:
        df = df.merge(aqi_annual.rename(columns={'zip':'ZCTA5'}), on="ZCTA5", how="left")
    if hhi is not None and not hhi.empty:
        df = df.merge(hhi.rename(columns={'zip':'ZCTA5'}), on="ZCTA5", how="left")
    stamp(f"Merged shape: {df.shape}")

# ---------- Feature engineering ----------
with Step("Feature engineering"):
    # Rename to friendly
    df["median_income"]   = pd.to_numeric(df["S1901_C01_012E"], errors="coerce")
    df["poor_health_pct"] = pd.to_numeric(df["GHLTH_CrudePrev"], errors="coerce")
    df["pharmacies_count"]= pd.to_numeric(df["pharmacies_count"], errors="coerce").fillna(0)

    # Impute income & health with national medians
    for col in ["median_income","poor_health_pct"]:
        med = df[col].median(skipna=True)
        df[col] = df[col].fillna(med)

    # Access: population-based scarcity (people per pharmacy; higher = worse)
    denom = df["pharmacies_count"].replace(0, 1)  # keep finite for 0-pharmacy ZIPs
    df["pop_per_pharmacy"] = np.where(df["population"].gt(0), df["population"] / denom, np.nan)

    # Density (optional; convert ALAND sq meters to km2 if present)
    if "land_area_raw" in df.columns:
        med_area = np.nanmedian(df["land_area_raw"])
        if np.isfinite(med_area) and med_area > 1e5:
            df["land_area_km2"] = df["land_area_raw"] / 1e6
        else:
            df["land_area_km2"] = df["land_area_raw"]
        df["pop_density"] = np.where(df["land_area_km2"].gt(0), df["population"]/df["land_area_km2"], np.nan)
    else:
        df["pop_density"] = np.nan

    # AQI/HHI numeric
    if "aqi" in df.columns:       df["aqi"] = pd.to_numeric(df["aqi"], errors="coerce")
    if "heat_hhb" in df.columns:  df["heat_hhb"] = pd.to_numeric(df["heat_hhb"], errors="coerce")

    # Percentile features (0..1, higher = worse)
    df["income_pct_inv"]  = 1 - pct_rank(df["median_income"])
    df["health_pct"]      = pct_rank(df["poor_health_pct"])
    df["access_pct_inv"]  = pct_rank(df["pop_per_pharmacy"])
    df["density_pct"]     = pct_rank(df["pop_density"]) if df["pop_density"].notna().any() else 0.5
    df["aqi_pct"]         = pct_rank(df["aqi"])         if "aqi" in df.columns else 0.5
    df["heat_pct"]        = pct_rank(df["heat_hhb"])    if "heat_hhb" in df.columns else 0.5

# ---------- Composite + IsolationForest ----------
with Step("Compute composite & train IsolationForest"):
    # Coverage checks (fraction of non-null)
    cov = {
        'aqi':    ("aqi" in df.columns) and df["aqi"].notna().mean(),
        'heat':   ("heat_hhb" in df.columns) and df["heat_hhb"].notna().mean(),
        'density': df["pop_density"].notna().mean()
    }
    print("Coverage:", cov)

    use_aqi  = bool(cov["aqi"])  and (cov["aqi"]  >= 0.30)
    use_heat = bool(cov["heat"]) and (cov["heat"] >= 0.30)
    use_dens = bool(cov["density"]) and (cov["density"] >= 0.20)

    # Base weights (sum ~1 before normalization)
    w_income, w_health, w_access = 0.30, 0.35, 0.30
    w_aqi   = 0.05 if use_aqi  else 0.00
    w_heat  = 0.05 if use_heat else 0.00
    w_dens  = 0.00 if not use_dens else 0.02  # keep small even if available

    # Normalize to sum=1
    Wsum = w_income + w_health + w_access + w_aqi + w_heat + w_dens
    w_income, w_health, w_access, w_aqi, w_heat, w_dens = [w/Wsum for w in [w_income, w_health, w_access, w_aqi, w_heat, w_dens]]

    # Composite (transparent)
    df["composite"] = (
        w_income * df["income_pct_inv"] +
        w_health * df["health_pct"] +
        w_access * df["access_pct_inv"] +
        w_aqi    * (df["aqi_pct"]   if use_aqi  else 0) +
        w_heat   * (df["heat_pct"]  if use_heat else 0) +
        w_dens   * (df["density_pct"] if use_dens else 0)
    )

    # IF features = same percentile vector
    feat_cols = ["income_pct_inv","health_pct","access_pct_inv"]
    if use_aqi:  feat_cols.append("aqi_pct")
    if use_heat: feat_cols.append("heat_pct")
    if use_dens: feat_cols.append("density_pct")
    X = df[feat_cols].fillna(0.5).to_numpy()

    iforest = IsolationForest(random_state=42, contamination=CONTAMINATION, n_estimators=150, n_jobs=-1)
    iforest.fit(X)
    anom = -iforest.decision_function(X)                     # invert so higher = more anomalous
    anom = (anom - anom.min()) / (anom.max() - anom.min() + 1e-12)  # 0..1
    df["iforest_anomaly"] = anom

    # Final score = blend
    df["IFAE_score"] = 0.5*df["composite"] + 0.5*df["iforest_anomaly"]

# ---------- Finalize outputs ----------
with Step("Finalize outputs"):
    out = Path(OUT_DIR); out.mkdir(parents=True, exist_ok=True)

    keep = [
        "ZCTA5","IFAE_score","composite","iforest_anomaly",
        "median_income","poor_health_pct","population","pharmacies_count",
        "pop_per_pharmacy","income_pct_inv","health_pct","access_pct_inv",
        "density_pct","pop_density"
    ]
    if "aqi" in df.columns:       keep += ["aqi","aqi_pct","obs_total"]
    if "heat_hhb" in df.columns:  keep += ["heat_hhb","heat_pct"]
    if "land_area_km2" in df.columns: keep += ["land_area_km2"]

    ranked = df[keep].copy().sort_values("IFAE_score", ascending=False).reset_index(drop=True)

    # Save everything
    out_full   = out / "national_ifae_rank.csv"
    out_top    = out / "top5_ifae.csv"
    out_bottom = out / "bottom5_ifae.csv"
    ranked.to_csv(out_full, index=False)

    # Population gate for top-k presentation
    eligible = ranked[ ranked["population"].fillna(0) >= MIN_POP_TOPK ]
    top_presentable = eligible.head(TOP_K)
    bottom_presentable = ranked.tail(TOP_K)

    top_presentable.to_csv(out_top, index=False)
    bottom_presentable.to_csv(out_bottom, index=False)

    stamp(f"Wrote {out_full}")
    stamp(f"Wrote {out_top} (pop ≥ {MIN_POP_TOPK})")
    stamp(f"Wrote {out_bottom}")

    print("\nTop 5 (pop ≥ {:,}) preview:\n".format(MIN_POP_TOPK), top_presentable.head(5).to_string(index=False))
    print("\nBottom 5 preview:\n", bottom_presentable.tail(5).to_string(index=False))

stamp("All done. 🚀")


[08:49:24 +  0.00s] ▶ Load FINANCIAL (income) ...
[08:49:24 +  0.32s] FIN rows=33773, null income=3264
[08:49:24 +  0.32s] ✓ Load FINANCIAL (income) done in 0.32s
[08:49:24 +  0.32s] ▶ Load HEALTH (poor general health %) ...
[08:49:24 +  0.53s] HEALTH rows=32520, null health%=0
[08:49:24 +  0.53s] ✓ Load HEALTH (poor general health %) done in 0.21s
[08:49:24 +  0.53s] ▶ Load PHARMACY (access) ...
[08:49:25 +  1.20s] PHARMACY unique ZCTA5=14940, pharmacies=61970
[08:49:25 +  1.20s] ✓ Load PHARMACY (access) done in 0.67s
[08:49:25 +  1.20s] ▶ Load POPULATION (skiprows=10 + autodetect) ...
[08:49:25 +  1.26s] POP rows=40959, null pop=0, land_col=None
[08:49:25 +  1.26s] ✓ Load POPULATION (skiprows=10 + autodetect) done in 0.06s
[08:49:25 +  1.26s] ▶ Load AQI (annual weighted PM2.5) ...
[08:49:26 +  2.85s] AQI rows=125 (coverage ~0.3%)
[08:49:26 +  2.85s] ✓ Load AQI (annual weighted PM2.5) done in 1.59s
[08:49:26 +  2.85s] ▶ Load HHI (heat vulnerability) ...


/var/folders/9d/5gzqwh3d5sx43n9fsy97czyc0000gn/T/ipykernel_54177/2731130997.py:98: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grp_annual = df.groupby('zip', as_index=False).apply(


[08:49:33 +  9.77s] HHI rows=32092
[08:49:33 +  9.77s] ✓ Load HHI (heat vulnerability) done in 6.93s
[08:49:33 +  9.77s] ▶ Merge all features by ZCTA/ZIP ...
[08:49:33 +  9.81s] Merged shape: (41094, 10)
[08:49:33 +  9.82s] ✓ Merge all features by ZCTA/ZIP done in 0.04s
[08:49:33 +  9.82s] ▶ Feature engineering ...
[08:49:33 +  9.83s] ✓ Feature engineering done in 0.02s
[08:49:33 +  9.83s] ▶ Compute composite & train IsolationForest ...
Coverage: {'aqi': np.float64(0.003041806589769796), 'heat': np.float64(0.7806979121039568), 'density': np.float64(0.0)}
[08:49:34 + 10.25s] ✓ Compute composite & train IsolationForest done in 0.42s
[08:49:34 + 10.25s] ▶ Finalize outputs ...
[08:49:34 + 10.46s] Wrote results/national_ifae_rank.csv
[08:49:34 + 10.46s] Wrote results/top5_ifae.csv (pop ≥ 1000)
[08:49:34 + 10.46s] Wrote results/bottom5_ifae.csv

Top 5 (pop ≥ 1,000) preview:
 ZCTA5  IFAE_score  composite  iforest_anomaly  median_income  poor_health_pct  population  pharmacies_count  pop_per_p